In [2]:
import pandas as pd
import re

In [3]:
file_indigo = r"/home/xena/Desktop/prateek/oneGo/onego_poc1/files/flight_schedules/FlightSchedule_Indigo.xlsx"

schedule_indigo = pd.read_excel(file_indigo)

In [4]:
schedule_indigo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107001 entries, 0 to 107000
Data columns (total 10 columns):
Origin               107001 non-null object
Destination          107001 non-null object
6E FlightNumber      107001 non-null object
Departure(LT)        107001 non-null object
Arrival(LT)          107001 non-null object
Flight Type          107001 non-null object
Via                  107001 non-null object
Days Of Operation    107001 non-null object
Effective From       107001 non-null object
Effective To         107001 non-null object
dtypes: object(10)
memory usage: 8.2+ MB


In [5]:
def separate_code(x):
    list = str(x).split('(')
    code = list[-1].rstrip(')').strip(' ')
    name = list[0].strip(' ')
#     print(name)
    return (name,code)

In [6]:
schedule_indigo.loc[:,'Origin_Code'] = schedule_indigo['Origin'].apply(lambda x:separate_code(x)[1])
schedule_indigo.loc[:,'Origin_Name'] = schedule_indigo['Origin'].apply(lambda x:separate_code(x)[0])

schedule_indigo.loc[:,'Destination_Code'] = schedule_indigo['Destination'].apply(lambda x:separate_code(x)[1])
schedule_indigo.loc[:,'Destination_Name'] = schedule_indigo['Destination'].apply(lambda x:separate_code(x)[0])

In [22]:
schedule_indigo.head()

,Origin,Destination,6E FlightNumber,Departure(LT),Arrival(LT),Flight Type,Via,Days Of Operation,Effective From,Effective To,Origin_Code,Origin_Name,Destination_Code,Destination_Name
0,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 892,23:30,15:45,Connecting,BOM/CCU,Daily,12-Jul-19,25-Oct-19,AUH,Abu Dhabi,IXA,Agartala
1,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 6749/6E 892,23:30,15:45,Connecting,BOM/CCU,Daily,12-Jul-19,25-Oct-19,AUH,Abu Dhabi,IXA,Agartala
2,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 374,23:30,18:50,Connecting,BOM/CCU,Except TUE,15-Jun-19,06-Aug-19,AUH,Abu Dhabi,IXA,Agartala
3,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 276,23:30,17:40,Connecting,BOM/CCU,Except TUE,15-Jun-19,20-Jul-19,AUH,Abu Dhabi,IXA,Agartala
4,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 798,23:30,14:50,Connecting,BOM/CCU,Except TUE,15-Jun-19,25-Oct-19,AUH,Abu Dhabi,IXA,Agartala


In [25]:
schedule_indigo.rename(columns = {'Flight Type': 'Flight_Type'}, inplace =True)
direct = schedule_indigo[schedule_indigo.Flight_Type == 'Direct']